In [17]:
import numpy as np
import pandas as pd
import os
from function import Slope, minSSE_recovery

s_c = '結算價'
top_path = './../../../'
Data_path = top_path + 'InterpData/'
expiry = 'NearbyMonth'

IV_type = 'putIV'
K_Range = [500, 300]
K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])
IV_path = '{}/{}/{}/{}'.format(Data_path, expiry, IV_type, K_Range_file)

Dir_tree = [top_path, 'ForecastData', expiry, IV_type, K_Range_dir]
current_path = Dir_tree[0]
for i in range(1, len(Dir_tree), 1):
    if Dir_tree[i] not in os.listdir(current_path):
        os.mkdir(current_path + Dir_tree[i])
    current_path = current_path + Dir_tree[i] + '/'

IV_data = pd.read_csv(IV_path, encoding='Big5', index_col=False)
IV_matrix = np.array(IV_data)

#----------------------------------------------------------------------------


OC = np.array(IV_data['期貨開盤價'] - IV_data['期貨收盤價'])
HL = np.array(IV_data['期貨最高價'] - IV_data['期貨最低價'])
expirty_days = np.array(IV_data['到期天數'])
#OC_HL_K_E = np.vstack((OC, HL, Kmin_reduce_F, Kmax_reduce_F, expirty_days)).T
OCHL = np.vstack((OC, HL)).T


IV_matrix = np.array(IV_data)
K_num = len(np.where(IV_matrix[0, 0] == IV_matrix)[0])
K = np.array(IV_data['履約價'])
K = np.reshape(K, (-1, K_num))
IV= np.array(IV_data['隱含波動率({})'.format(s_c)])
IV = np.reshape(IV, (-1, K_num))
K_IVslope = Slope(X=K, Y=IV, axis=1)
E = IV_matrix[range(0, len(IV_matrix), K_num), IV_data.columns.get_loc('到期天數')]

In [18]:
one_day_expiry_idx = np.arange(len(E))[np.equal(E, 1)]
if one_day_expiry_idx[-1] == len(E)-1:
    one_day_expiry_idx = one_day_expiry_idx[:-1]
most_days_expiry_idx = one_day_expiry_idx + 1
most_days_expiry_idx = np.hstack((0, most_days_expiry_idx))

most_days_expiry_idx0 = most_days_expiry_idx[:-1]
most_days_expiry_idx1 = most_days_expiry_idx[1:]
trade_days_in_month = most_days_expiry_idx1 - most_days_expiry_idx0  
most_days_expiry = E[most_days_expiry_idx]

contract_appear_days1 = [list(range(1, trade_day_in_month+1, 1)) \
                for trade_day_in_month in trade_days_in_month]
contract_appear_days1 = [contract_appear_day1 for subcontract_appear_day1 in contract_appear_days1\
                         for contract_appear_day1 in subcontract_appear_day1]
contract_appear_days1 = np.array(contract_appear_days1)
contract_appear_days2 = np.arange(1, len(E) - most_days_expiry_idx[-1] +1, 1)
contract_appear_days = np.hstack((contract_appear_days1, contract_appear_days2))
#contract_appear_days 為該契約(特定交易日期、到期日期，履約價不限)上市的交易日數，例如第一天上市到期天數35天，
#則值為1，第二天則到期日為34天值為2，一直到k(因為是交易日數，所以數字不一定)。然後又到下個月的契約，值又從1開
#始
from function import TimeSeriesData
seq_length1 = 30
seq_length2 = seq_length1 + 1
min_contract_day = 2
magnification_slope = 100



#變數有考慮到期日
#K_IVslope_E = np.hstack((K_IVslope*magnification_slope, np.reshape(E, (-1, 1))))

#Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, \
                                      #drop_out_columns=[len(K_IVslope_E[0])-1])

#變數沒考慮到期日
K_IVslope_E = K_IVslope * magnification_slope
Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, drop_out_columns=[])



Inputs_OCHLE  = OCHL[range(0, len(IV_matrix), K_num)]
Inputs_OCHLE  = np.array([Inputs_OCHLE[i:i+seq_length2] \
                              for i in range(len(Inputs_OCHLE)-seq_length2+1)])
Inputs_OCHLE  = Inputs_OCHLE.astype(float)

Inputs_slope = np.reshape(Inputs_slope, (Inputs_slope.shape[0], Inputs_slope.shape[1] * Inputs_slope.shape[2]))
Inputs_OCHLE = np.reshape(Inputs_OCHLE, (Inputs_OCHLE.shape[0], Inputs_OCHLE.shape[1] * Inputs_OCHLE.shape[2]))


Inputs_OCHLE  = Inputs_OCHLE.astype(float)
if seq_length1  > seq_length2-1:
    Inputs_OCHLE = Inputs_OCHLE[seq_length1-seq_length2+1:]
    contract_appear_days = contract_appear_days[seq_length1-seq_length2+1:]
    IV_matrix_forecast = IV_matrix[seq_length1*K_num:]
if seq_length1 <= seq_length2-1:
    Inputs_slope = Inputs_slope[seq_length2-1-seq_length1:]
    Ouputs = Ouputs[seq_length2-1-seq_length1:]
    contract_appear_days = contract_appear_days[seq_length2-1:]
    IV_matrix_forecast = IV_matrix[(seq_length2-1)*K_num:]
Inputs = np.hstack((Inputs_slope, Inputs_OCHLE))

match_cond = np.greater_equal(contract_appear_days, min_contract_day)
Inputs = Inputs[match_cond]
contract_appear_days_filter = contract_appear_days[match_cond]
contract_appear_days_filter_Knum = contract_appear_days_filter.repeat(K_num)
match_cond_Knum = match_cond.repeat(K_num)
IV_matrix_forecast = IV_matrix_forecast[match_cond_Knum]
IV_matrix_forecast= np.hstack((IV_matrix_forecast, np.reshape(contract_appear_days_filter_Knum, (-1, 1))))
Ouputs = Ouputs[match_cond]



train_size = int(len(Inputs)*0.8)
#val_size = int(train_size*0.2)
X_train = Inputs[:train_size]
y_train = Ouputs[:train_size]
#y_train = Ouputs[:train_size-val_size]
#y_val = Ouputs[train_size - val_size:train_size]


X_test = Inputs[train_size:]
y_test = Ouputs[train_size:]
IV_matrix_test = IV_matrix_forecast[K_num*train_size:]


In [19]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt.acquisition import gaussian_ei

#進行調參的次數
n_calls=20
#此函式可以幫我們設定一個模型的結構
def set_Linear_series(model_structure):
    neurons_num = model_structure['neurons_num']
    #activations = model_structure['dense_activations']
    kernel_regularizer= model_structure['kernel_regularizer']

    input = Input(shape=model_structure['input_shape'])
    hidden = input

    for i in range(0, len(neurons_num), 1):
        hidden = Dense(neurons_num[i], kernel_regularizer=kernel_regularizer[i])(hidden)
    output = hidden

    model = Model(inputs= input, outputs=output)
    return model

search_space = [
    Real(1e-6, 1e-2, prior='log-uniform', name='lambda')
]


def train_and_evaluate_net(neurons_num_, layers_num, lambda_):
    batch_size = 500
    epochs = 1000

    neurons_num = [neurons_num_]* layers_num 
    kernel_regularizer=[l2(lambda_)]*len(neurons_num)

    model_structure = {
        'input_shape': X_train.shape[1],
        'neurons_num': neurons_num,
        'kernel_regularizer':kernel_regularizer
        }

    alltrain = 1
    allMSE = np.zeros(alltrain) 
    for j in range(alltrain):
        #設定模型結構
        model = set_Linear_series(model_structure)
        model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

        # 訓練模型
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, \
                                verbose=2, mode='min', restore_best_weights=True)
        hist_model = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, \
            validation_split=0.2, callbacks=[early_stopping]
        )

        #得到輸出
        y_pred = model.predict(X_test)
        y_pred = y_pred / magnification_slope
        columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
        column_index = [IV_data.columns.get_loc(col) for col in columns_names]

        SSE_everyday = np.zeros(((len(y_pred), K_num)))
    
        for i in range(len(y_pred)):
            K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
            iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
            y_pred_day = y_pred[i]

            ForecastData, SSE_everyday[i] = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        sse_everyday = SSE_everyday[:, 0]
        allMSE[j] = np.mean(sse_everyday)

    loss = np.min(allMSE)
    return loss

@use_named_args(search_space)
def objective(**params):
    lambda_ = params['lambda']
    
    loss = train_and_evaluate_net(neurons_num_=y_train.shape[1], layers_num=1, lambda_=lambda_)
    return loss


result = gp_minimize(objective, search_space, n_calls=n_calls, random_state=42, acq_func='EI')

# 输出最佳参数
print("最佳参数: {}".format(result.x))
print("最小损失: {}".format(result.fun))


best_params = result.x
lambda_ = best_params[0]

Epoch 1/1000
4/4 [==============================] - 1s 80ms/step - loss: 228.3806 - mse: 228.0874 - val_loss: 590.5791 - val_mse: 590.2872
Epoch 2/1000
4/4 [==============================] - 0s 30ms/step - loss: 113.2082 - mse: 112.9169 - val_loss: 394.1989 - val_mse: 393.9097
Epoch 3/1000
4/4 [==============================] - 0s 38ms/step - loss: 83.2704 - mse: 82.9829 - val_loss: 343.1093 - val_mse: 342.8264
Epoch 4/1000
4/4 [==============================] - 0s 17ms/step - loss: 74.1136 - mse: 73.8337 - val_loss: 299.7569 - val_mse: 299.4838
Epoch 5/1000
4/4 [==============================] - 0s 16ms/step - loss: 63.2313 - mse: 62.9619 - val_loss: 241.7128 - val_mse: 241.4514
Epoch 6/1000
4/4 [==============================] - 0s 16ms/step - loss: 50.0843 - mse: 49.8268 - val_loss: 192.5220 - val_mse: 192.2723
Epoch 7/1000
4/4 [==============================] - 0s 18ms/step - loss: 40.3079 - mse: 40.0619 - val_loss: 161.5526 - val_mse: 161.3138
Epoch 8/1000
4/4 [===================

c:\Users\bgpa4\My paper N\.venv\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-06] before, using random point [0.0004426669795153249]
  warnings.warn(


4/4 [==============================] - 0s 66ms/step - loss: 218.1809 - mse: 218.0959 - val_loss: 574.3189 - val_mse: 574.2333
Epoch 2/1000
4/4 [==============================] - 0s 24ms/step - loss: 109.3552 - mse: 109.2692 - val_loss: 392.4438 - val_mse: 392.3572
Epoch 3/1000
4/4 [==============================] - 0s 27ms/step - loss: 82.8732 - mse: 82.7864 - val_loss: 346.5168 - val_mse: 346.4300
Epoch 4/1000
4/4 [==============================] - 0s 25ms/step - loss: 73.3899 - mse: 73.3033 - val_loss: 290.9361 - val_mse: 290.8500
Epoch 5/1000
4/4 [==============================] - 0s 25ms/step - loss: 59.6840 - mse: 59.5983 - val_loss: 229.8447 - val_mse: 229.7598
Epoch 6/1000
4/4 [==============================] - 0s 21ms/step - loss: 46.9283 - mse: 46.8438 - val_loss: 189.6857 - val_mse: 189.6019
Epoch 7/1000
4/4 [==============================] - 0s 18ms/step - loss: 39.2140 - mse: 39.1304 - val_loss: 163.0320 - val_mse: 162.9489
Epoch 8/1000
4/4 [==============================] 

c:\Users\bgpa4\My paper N\.venv\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-06] before, using random point [0.0005509505464468562]
  warnings.warn(


4/4 [==============================] - 1s 62ms/step - loss: 224.5969 - mse: 224.4908 - val_loss: 589.7011 - val_mse: 589.5945
Epoch 2/1000
4/4 [==============================] - 0s 39ms/step - loss: 114.6717 - mse: 114.5647 - val_loss: 403.3086 - val_mse: 403.2011
Epoch 3/1000
4/4 [==============================] - 0s 35ms/step - loss: 85.3422 - mse: 85.2347 - val_loss: 343.5921 - val_mse: 343.4848
Epoch 4/1000
4/4 [==============================] - 0s 24ms/step - loss: 73.6513 - mse: 73.5443 - val_loss: 288.4743 - val_mse: 288.3682
Epoch 5/1000
4/4 [==============================] - 0s 23ms/step - loss: 61.1317 - mse: 61.0261 - val_loss: 233.2360 - val_mse: 233.1315
Epoch 6/1000
4/4 [==============================] - 0s 20ms/step - loss: 49.4334 - mse: 49.3294 - val_loss: 191.2690 - val_mse: 191.1661
Epoch 7/1000
4/4 [==============================] - 0s 23ms/step - loss: 40.8421 - mse: 40.7396 - val_loss: 159.8310 - val_mse: 159.7294
Epoch 8/1000
4/4 [==============================] 

...

4/4 [==============================] - 1s 76ms/step - loss: 242.2445 - mse: 242.2231 - val_loss: 625.6772 - val_mse: 625.6558
Epoch 2/1000
4/4 [==============================] - 0s 34ms/step - loss: 119.9152 - mse: 119.8935 - val_loss: 404.7245 - val_mse: 404.7026
Epoch 3/1000
4/4 [==============================] - 0s 28ms/step - loss: 85.1442 - mse: 85.1222 - val_loss: 344.7143 - val_mse: 344.6921
Epoch 4/1000
4/4 [==============================] - 0s 24ms/step - loss: 75.1533 - mse: 75.1311 - val_loss: 301.1571 - val_mse: 301.1349
Epoch 5/1000
4/4 [==============================] - 0s 22ms/step - loss: 64.3975 - mse: 64.3753 - val_loss: 245.6528 - val_mse: 245.6307
Epoch 6/1000
4/4 [==============================] - 0s 22ms/step - loss: 51.9118 - mse: 51.8897 - val_loss: 197.3621 - val_mse: 197.3401
Epoch 7/1000
4/4 [==============================] - 0s 25ms/step - loss: 42.0074 - mse: 41.9853 - val_loss: 163.9212 - val_mse: 163.8992
Epoch 8/1000
4/4 [==============================] 

c:\Users\bgpa4\My paper N\.venv\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-06] before, using random point [0.0008945717026779293]
  warnings.warn(


4/4 [==============================] - 1s 74ms/step - loss: 196.6811 - mse: 196.5098 - val_loss: 549.2327 - val_mse: 549.0616
Epoch 2/1000
4/4 [==============================] - 0s 32ms/step - loss: 110.2501 - mse: 110.0792 - val_loss: 403.9366 - val_mse: 403.7665
Epoch 3/1000
4/4 [==============================] - 0s 38ms/step - loss: 85.1876 - mse: 85.0183 - val_loss: 332.4209 - val_mse: 332.2537
Epoch 4/1000
4/4 [==============================] - 0s 35ms/step - loss: 70.3732 - mse: 70.2072 - val_loss: 274.5625 - val_mse: 274.3992
Epoch 5/1000
4/4 [==============================] - 0s 24ms/step - loss: 58.2476 - mse: 58.0856 - val_loss: 226.9036 - val_mse: 226.7444
Epoch 6/1000
4/4 [==============================] - 0s 25ms/step - loss: 47.9756 - mse: 47.8178 - val_loss: 185.2779 - val_mse: 185.1228
Epoch 7/1000
4/4 [==============================] - 0s 23ms/step - loss: 39.1750 - mse: 39.0212 - val_loss: 150.6657 - val_mse: 150.5144
Epoch 8/1000
4/4 [==============================] 

c:\Users\bgpa4\My paper N\.venv\lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1e-06] before, using random point [0.007671161335380119]
  warnings.warn(


4/4 [==============================] - 1s 70ms/step - loss: 196.0483 - mse: 194.6126 - val_loss: 539.8199 - val_mse: 538.4629
Epoch 2/1000
4/4 [==============================] - 0s 27ms/step - loss: 108.0688 - mse: 106.7567 - val_loss: 389.0032 - val_mse: 387.7868
Epoch 3/1000
4/4 [==============================] - 0s 30ms/step - loss: 83.6756 - mse: 82.5081 - val_loss: 333.1398 - val_mse: 332.0742
Epoch 4/1000
4/4 [==============================] - 0s 30ms/step - loss: 71.8416 - mse: 70.8257 - val_loss: 274.5861 - val_mse: 273.6707
Epoch 5/1000
4/4 [==============================] - 0s 29ms/step - loss: 58.0285 - mse: 57.1595 - val_loss: 216.6283 - val_mse: 215.8508
Epoch 6/1000
4/4 [==============================] - 0s 29ms/step - loss: 46.1148 - mse: 45.3780 - val_loss: 177.9469 - val_mse: 177.2895
Epoch 7/1000
4/4 [==============================] - 0s 29ms/step - loss: 38.5830 - mse: 37.9602 - val_loss: 152.2327 - val_mse: 151.6767
Epoch 8/1000
4/4 [==============================] 

In [20]:
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import os
import json
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

def convert_to_serializable(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    elif isinstance(obj, l2):
        return {"type": "L2", "l2": obj.l2}
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")
#決定這次的模型編號
model_idx = 1
#本次預測的變數為：slope(隱波與履約價之間的斜率)或iv(隱波的數值)
forecast_variable = 'slope'
#資料設定都相同的情況下，最多儲存多少種模型結果
max_model = 20

batch_size = 500
epochs = 1000
model_structure = {
        'input_shape': X_test.shape[1],
        'neurons_num': [y_test.shape[1]],
        'kernel_regularizer':[l2(lambda_)]
    }

structure_list = [model_structure]
structure_name_list = ['model_structure']
General_structure = {
    structure_name_list[0] : structure_list[0],
}


def save_model_txt(file_path, structure_list, structure_name_list, mse, mse_adj):
    with open('{}.txt'.format(file_path), 'w', encoding='utf-8') as txt_file:
        for i in range(len(structure_list)):
            txt_file.write(structure_name_list[i] + ':\n')
            for key, value in structure_list[i].items():
                txt_file.write(f"{key}: {value}\n")
            txt_file.write('-----------------\n')
        txt_file.write('MSE:{}'.format(mse))
        txt_file.write('MSE_adj:{}'.format(mse_adj))


alltrain = 10
allMSE = np.zeros(alltrain)
allMSE_adj = np.zeros(alltrain)  
allForecastIV = np.zeros((K_num*len(X_test), alltrain))
allSSE_everyday = np.zeros((K_num*len(X_test), alltrain))
allSSE_everyday_adj = np.zeros((K_num*len(X_test), alltrain))
for j in range(alltrain):
    #設定模型結構
    model = set_Linear_series(model_structure)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

    # 訓練模型
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, \
                               verbose=2, mode='min', restore_best_weights=True)
    hist_model = model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, \
        validation_split=0.2, callbacks=[early_stopping]
    )




    #得到輸出
    y_pred = model.predict(X_test)
    y_pred = y_pred / magnification_slope
    columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
    column_index = [IV_data.columns.get_loc(col) for col in columns_names]
    ForecastIV = np.zeros(((len(y_pred), K_num)))
    SSE_everyday = np.zeros(((len(y_pred), K_num)))
    SSE_everyday_adj = np.zeros(((len(y_pred), K_num)))

    model_name = 'model{}'.format(model_idx)
    #model_file = 'model{}.h5'.format(model_idx)
    model_png = 'model{}.png'.format(model_idx)
    Forecast_name = 'model{}.csv'.format(model_idx)
    model_txt = 'model{}.txt'.format(model_idx)
    model_json = 'model{}.json'.format(model_idx)
    for i in range(len(y_pred)):
        K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
        iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
        y_pred_day = y_pred[i]
        iv_mean = iv_day / np.mean(iv_day)
        ForecastIV[i], sse_day = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        SSE_everyday[i] = sse_day
        SSE_everyday_adj[i] = sse_day / iv_mean

    ForecastIV = np.reshape(ForecastIV,(-1,1))
    sse_everyday = SSE_everyday[:, 0]
    sse_everyday_adj = SSE_everyday_adj[:, 0]
    SSE_everyday = np.reshape(SSE_everyday ,(-1,1))
    SSE_everyday_adj = np.reshape(SSE_everyday_adj ,(-1,1))
    allMSE[j] = np.mean(sse_everyday)
    allMSE_adj[j] = np.mean(sse_everyday_adj)
    allForecastIV[:,j:j+1] = ForecastIV
    allSSE_everyday[:, j:j+1] = SSE_everyday
    allSSE_everyday_adj[:, j:j+1] = SSE_everyday_adj

min_MSE_idx = np.argsort(allMSE)[0]
ForecastIV = allForecastIV[:, min_MSE_idx:min_MSE_idx+1]
SSE_everyday = allSSE_everyday[:, min_MSE_idx:min_MSE_idx+1]
SSE_everyday_adj = allSSE_everyday_adj[:, min_MSE_idx:min_MSE_idx+1]
min_MSE = allMSE[min_MSE_idx]
min_MSE_adj = allMSE_adj[min_MSE_idx]

Forecast_matrix  = np.hstack((IV_matrix_test, ForecastIV, SSE_everyday, SSE_everyday_adj))
column = np.hstack((IV_data.columns.to_numpy(), \
                        np.array(['上市天數(交易日)','預測隱含波動率({})'.format(s_c), 'loss', '調整後loss'])))
    
   
Forecast_Data = pd.DataFrame(data=Forecast_matrix, columns=column)


loss_columns_names = ['交易日期', '到期天數', '上市天數(交易日)', 'loss']
loss_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_columns_names]
loss_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_columns_index]
loss_Data = loss_Data.reset_index().iloc[:,1:]
loss_Data = loss_Data.rename(columns={'loss': model_name})
MSE_data = pd.DataFrame(columns=loss_Data.columns, data=[['MSE', 'MSE', 'MSE', min_MSE]])
loss_Data = pd.concat([loss_Data, MSE_data], axis=0)
loss_Data = loss_Data.reset_index().iloc[:,1:]
print(min_MSE)




model_path = top_path
model_type = 'TimeLinear'

model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_seq{}_min{}'.format(seq_length1, seq_length2, min_contract_day), forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'

loss_adj_columns_names = ['交易日期', '到期天數', '調整後loss']
loss_adj_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_adj_columns_names]
loss_adj_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_adj_columns_index]
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]
loss_adj_Data = loss_adj_Data.rename(columns={'調整後loss': model_name})
MSE_adj_data = pd.DataFrame(columns=loss_adj_Data.columns, data=[['MSE', 'MSE', min_MSE_adj]])
loss_adj_Data = pd.concat([loss_adj_Data, MSE_adj_data], axis=0)
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]



if 'Modelloss.csv' in os.listdir(model_path):
    Modelloss = pd.read_csv(model_path + 'Modelloss.csv', index_col=False, encoding='Big5')
    Modelloss_adj = pd.read_csv(model_path + 'Modelloss_adj.csv', index_col=False, encoding='Big5')
    if model_name in Modelloss.columns:
        if Modelloss[model_name][len(Modelloss)-1] > min_MSE:
            Modelloss[model_name] = loss_Data[model_name]
            Modelloss_adj[model_name] = loss_adj_Data[model_name]
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            #model.save(model_path + model_file)
            with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
            save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    elif len(Modelloss.columns) >= max_model+2 :
        all_MSE = np.array(Modelloss.iloc[-1, 2:])
        max_MSE = np.max(all_MSE)
        if max_MSE > min_MSE:
            max_MSE_idx = (np.arange(len(all_MSE))[np.equal(all_MSE, max_MSE)])[0] +2
            Modelloss.iloc[:,max_MSE_idx] = loss_Data[model_name]
            Modelloss_adj.iloc[:,max_MSE_idx] = loss_adj_Data[model_name]
            model_name = Modelloss.columns[max_MSE_idx]
            Forecast_name = model_name + '.csv'
            #model_file = model_name + '.h5'
            model_png = model_name + '.png'
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            #model.save(model_path + model_file)
            with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
            save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    else:
        Modelloss = pd.concat([Modelloss, loss_Data[model_name]], axis=1)
        Modelloss = Modelloss.reset_index().iloc[:,1:] 
        Modelloss_adj = pd.concat([Modelloss_adj, loss_Data[model_name]], axis=1)
        Modelloss_adj = Modelloss_adj.reset_index().iloc[:,1:] 
        Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
        #model.save(model_path + model_file)
        with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
        save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
        plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
        print('此模型已經被儲存為{}'.format(model_name))
else:
    Modelloss = loss_Data
    Modelloss_adj = loss_adj_Data
    Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
    #model.save(model_path + model_file)
    with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
    save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
    plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)

    
Modelloss.to_csv(model_path + 'Modelloss.csv', index=False, encoding='Big5')
Modelloss_adj.to_csv(model_path + 'Modelloss_adj.csv', index=False, encoding='Big5')




Epoch 1/1000
4/4 [==============================] - 1s 92ms/step - loss: 207.4526 - mse: 207.4469 - val_loss: 568.3715 - val_mse: 568.3657
Epoch 2/1000
4/4 [==============================] - 0s 47ms/step - loss: 109.9301 - mse: 109.9243 - val_loss: 401.1992 - val_mse: 401.1933
Epoch 3/1000
4/4 [==============================] - 0s 39ms/step - loss: 83.9898 - mse: 83.9840 - val_loss: 339.1565 - val_mse: 339.1506
Epoch 4/1000
4/4 [==============================] - 0s 44ms/step - loss: 71.7412 - mse: 71.7353 - val_loss: 285.2202 - val_mse: 285.2143
Epoch 5/1000
4/4 [==============================] - 0s 41ms/step - loss: 59.4936 - mse: 59.4877 - val_loss: 228.7725 - val_mse: 228.7666
Epoch 6/1000
4/4 [==============================] - 0s 32ms/step - loss: 47.1773 - mse: 47.1714 - val_loss: 183.9011 - val_mse: 183.8951
Epoch 7/1000
4/4 [==============================] - 0s 24ms/step - loss: 38.4415 - mse: 38.4355 - val_loss: 154.3064 - val_mse: 154.3004
Epoch 8/1000
4/4 [===================

In [30]:
one_day_expiry_idx = np.arange(len(E))[np.equal(E, 1)]
if one_day_expiry_idx[-1] == len(E)-1:
    one_day_expiry_idx = one_day_expiry_idx[:-1]
most_days_expiry_idx = one_day_expiry_idx + 1
most_days_expiry_idx = np.hstack((0, most_days_expiry_idx))

most_days_expiry_idx0 = most_days_expiry_idx[:-1]
most_days_expiry_idx1 = most_days_expiry_idx[1:]
trade_days_in_month = most_days_expiry_idx1 - most_days_expiry_idx0  
most_days_expiry = E[most_days_expiry_idx]

contract_appear_days1 = [list(range(1, trade_day_in_month+1, 1)) \
                for trade_day_in_month in trade_days_in_month]
contract_appear_days1 = [contract_appear_day1 for subcontract_appear_day1 in contract_appear_days1\
                         for contract_appear_day1 in subcontract_appear_day1]
contract_appear_days1 = np.array(contract_appear_days1)
contract_appear_days2 = np.arange(1, len(E) - most_days_expiry_idx[-1] +1, 1)
contract_appear_days = np.hstack((contract_appear_days1, contract_appear_days2))
#contract_appear_days 為該契約(特定交易日期、到期日期，履約價不限)上市的交易日數，例如第一天上市到期天數35天，
#則值為1，第二天則到期日為34天值為2，一直到k(因為是交易日數，所以數字不一定)。然後又到下個月的契約，值又從1開
#始
from function import TimeSeriesData
seq_length1 = 7
seq_length2 = seq_length1 + 1
min_contract_day = 2
magnification_slope = 10



K_IVslope_E = np.hstack((IV*magnification_slope, np.reshape(E, (-1, 1))))
#K_IVslope_E = K_IVslope * magnification_slope
Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, \
                                      drop_out_columns=[len(K_IVslope_E[0])-1])


Inputs_OCHLE  = OCHL[range(0, len(IV_matrix), K_num)]
Inputs_OCHLE  = np.array([Inputs_OCHLE[i:i+seq_length2] \
                              for i in range(len(Inputs_OCHLE)-seq_length2+1)])
Inputs_OCHLE  = Inputs_OCHLE.astype(float)

Inputs_slope = np.reshape(Inputs_slope, (Inputs_slope.shape[0], Inputs_slope.shape[1] * Inputs_slope.shape[2]))
Inputs_OCHLE = np.reshape(Inputs_OCHLE, (Inputs_OCHLE.shape[0], Inputs_OCHLE.shape[1] * Inputs_OCHLE.shape[2]))

Inputs = np.hstack((Inputs_slope, Inputs_OCHLE))

contract_appear_days = contract_appear_days[seq_length1:]
IV_matrix_forecast = IV_matrix[seq_length1*K_num:]


match_cond = np.greater_equal(contract_appear_days, min_contract_day)
Inputs = Inputs[match_cond]
contract_appear_days_filter = contract_appear_days[match_cond]
contract_appear_days_filter_Knum = contract_appear_days_filter.repeat(K_num)
match_cond_Knum = match_cond.repeat(K_num)
IV_matrix_forecast = IV_matrix_forecast[match_cond_Knum]
IV_matrix_forecast= np.hstack((IV_matrix_forecast, np.reshape(contract_appear_days_filter_Knum, (-1, 1))))
Ouputs = Ouputs[match_cond]



train_size = int(len(Inputs)*0.8)
#val_size = int(train_size*0.2)
X_train = Inputs[:train_size]
y_train = Ouputs[:train_size]
#y_train = Ouputs[:train_size-val_size]
#y_val = Ouputs[train_size - val_size:train_size]


X_test = Inputs[train_size:]
y_test = Ouputs[train_size:]
IV_matrix_test = IV_matrix_forecast[K_num*train_size:]


input_ = Input(shape=X_train.shape[1])
output_ = Dense(y_train.shape[1])(input_)
model = Model(inputs=input_, outputs=output_)
model.compile(loss='mse', optimizer = 'adam', metrics=['mse']) 
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, \
                               verbose=1, mode='min', restore_best_weights=True) 
hist_model = model.fit(
         X_train, y_train, epochs=epochs, batch_size=100, \
    validation_split=0.2, callbacks=[early_stopping]
    )




Epoch 1/500
18/18 [==============================] - 0s 9ms/step - loss: 138.6753 - mse: 138.6753 - val_loss: 319.8040 - val_mse: 319.8040
Epoch 2/500
18/18 [==============================] - 0s 4ms/step - loss: 66.8188 - mse: 66.8188 - val_loss: 225.3282 - val_mse: 225.3282
Epoch 3/500
18/18 [==============================] - 0s 5ms/step - loss: 41.7831 - mse: 41.7831 - val_loss: 153.7891 - val_mse: 153.7891
Epoch 4/500
18/18 [==============================] - 0s 5ms/step - loss: 28.0302 - mse: 28.0302 - val_loss: 105.5273 - val_mse: 105.5273
Epoch 5/500
18/18 [==============================] - 0s 5ms/step - loss: 19.7937 - mse: 19.7937 - val_loss: 78.0533 - val_mse: 78.0533
Epoch 6/500
18/18 [==============================] - 0s 4ms/step - loss: 14.2857 - mse: 14.2857 - val_loss: 57.7966 - val_mse: 57.7966
Epoch 7/500
18/18 [==============================] - 0s 5ms/step - loss: 10.6174 - mse: 10.6174 - val_loss: 42.8724 - val_mse: 42.8724
Epoch 8/500
18/18 [==========================

In [31]:
model_path = top_path
model_type = 'Time'
forecast_variable = 'iv'
model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), \
                  'days_{}_{}'.format(seq_length1, seq_length2), model_type, forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'

model_idx = 1
#while 'model{}.h5'.format(model_idx) in os.listdir(model_path):
    #model_idx + model_idx + 1
model_name = 'model{}'.format(model_idx)
model_file = 'model{}.h5'.format(model_idx)
model_png = 'model{}.png'.format(model_idx)
Forecast_name = 'model{}.csv'.format(model_idx)

#models = os.listdir
#test_loss = model.evaluate([OCHLE_test, X_test], y_test)
#print("Test Loss:", test_loss)
# 查看測試輸出
y_pred = model.predict(X_test)
columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
column_index = [IV_data.columns.get_loc(col) for col in columns_names]
ForecastIV = np.zeros(((len(y_pred), K_num)))
SSE_everyday = np.zeros(((len(y_pred), K_num)))
SSE_everyday_adj = np.zeros(((len(y_pred), K_num)))


for i in range(len(y_pred)):
    K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
    iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
    y_pred_day = y_pred[i] / magnification_slope
    c = (iv_day - y_pred_day).mean()
    iv_day_std = iv_day / np.std(iv_day, ddof=1)
    y_pred_day = y_pred_day + c
    y_pred_day_adj  =y_pred_day / np.std(iv_day, ddof=1)
    
    ForecastIV[i] = y_pred_day
    SSE_everyday[i] = np.sum((iv_day-y_pred) * (iv_day-y_pred))
    SSE_everyday_adj[i] = 1 / iv_day_std
    

ForecastIV = np.reshape(ForecastIV,(-1,1))
sse_everyday = SSE_everyday[:, 0]
sse_everyday_adj = SSE_everyday_adj[:, 0]
SSE_everyday = np.reshape(SSE_everyday ,(-1,1))
SSE_everyday_adj = np.reshape(SSE_everyday_adj ,(-1,1))
Forecast_matrix  = np.hstack((IV_matrix_test, ForecastIV, SSE_everyday, SSE_everyday_adj))
column = np.hstack((IV_data.columns.to_numpy(), np.array(['預測隱含波動率({})'.format(s_c), 'loss', '調整後loss'])))
#Forecast_Data = pd.DataFrame(data=Forecast_matrix, columns=column)


loss_columns_names = ['交易日期', '到期天數', 'loss']
loss_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_columns_names]
loss_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_columns_index]
loss_Data = loss_Data.reset_index().iloc[:,1:]
loss_Data = loss_Data.rename(columns={'loss': model_name})
MSE_data = pd.DataFrame(columns=loss_Data.columns, data=[['MSE', 'MSE', np.mean(sse_everyday)]])
loss_Data = pd.concat([loss_Data, MSE_data], axis=0)
loss_Data = loss_Data.reset_index().iloc[:,1:]
print(np.mean(sse_everyday))

#-----------------------------------------------

loss_adj_columns_names = ['交易日期', '到期天數', '調整後loss']
loss_adj_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_columns_names]
loss_adj_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_columns_index]
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]
loss_adj_Data = loss_adj_Data.rename(columns={'調整後loss': model_name})
MSE_adj_data = pd.DataFrame(columns=loss_adj_Data.columns, data=[['MSE', 'MSE', np.mean(sse_everyday_adj)]])
loss_adj_Data = pd.concat([loss_adj_Data, MSE_adj_data], axis=0)
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]


from tensorflow.keras.utils import plot_model
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

if 'Modelloss.csv' in os.listdir(model_path):
    Modelloss = pd.read_csv(model_path + 'Modelloss.csv', index_col=False, encoding='Big5')
    Modelloss_adj = pd.read_csv(model_path + 'Modelloss_adj.csv', index_col=False, encoding='Big5')
    if model_name in Modelloss.columns:
        if Modelloss[model_name][len(Modelloss)] > np.mean(sse_everyday):
            Modelloss[model_name] = loss_Data[model_name]
            Modelloss_adj[model_name] = loss_adj_Data[model_name]
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            model.save(model_path + model_file)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    elif len(Modelloss.columns) >= 22 :
        all_MSE = np.array(Modelloss.iloc[-1, 2:])
        max_MSE = np.max(all_MSE)
        if max_MSE > np.mean(sse_everyday):
            max_MSE_idx = (np.arange(len(all_MSE))[np.equal(all_MSE, max_MSE)])[0] +2
            Modelloss.iloc[:,max_MSE_idx] = loss_Data[model_name]
            Modelloss_adj.iloc[:,max_MSE_idx] = loss_adj_Data[model_name]
            model_name = Modelloss.columns[max_MSE_idx]
            Forecast_name = model_name + '.csv'
            model_file = model_name + '.h5'
            model_png = model_name + '.png'
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            model.save(model_path + model_file)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    else:
        Modelloss = pd.concat([Modelloss, loss_Data[model_name]], axis=1)
        Modelloss = Modelloss.reset_index().iloc[:,1:] 
        Modelloss_adj = pd.concat([Modelloss_adj, loss_Data[model_name]], axis=1)
        Modelloss_adj = Modelloss_adj.reset_index().iloc[:,1:] 
        Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
        model.save(model_path + model_file)
        plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)
        print('此模型已經被儲存為{}'.format(model_name))
else:
    Modelloss = loss_Data
    Modelloss_adj = loss_adj_Data
    Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
    model.save(model_path + model_file)
    plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=True)

    
Modelloss.to_csv(model_path + 'Modelloss.csv', index=False, encoding='Big5')
Modelloss_adj.to_csv(model_path + 'Modelloss_adj.csv', index=False, encoding='Big5')

18/18 [==============================] - 0s 987us/step
133207.37839625738


KeyError: 560